Устанавливаем необходимые библиотеки:

In [ ]:
from google.colab import files
!pip install pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.dataframe import DataFrame

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark.sql.functions import when

Загружаем датасеты:

In [ ]:
uploaded = files.upload()

Инициализируем спарк-сессию:

In [ ]:
spark = SparkSession.builder.getOrCreate()

Читаем датасет и сортируем его:

In [ ]:
initialDF = spark.read.parquet("/content/dataset")
initialDF = initialDF.select("*").orderBy("partner", "rep_date", "article")
initialDF.show()

+-------+----------+---------+---------+
|partner|  rep_date|    value|  article|
+-------+----------+---------+---------+
|  00000|2021-06-30|    -0.11|    begin|
|  00000|2021-06-30|     0.62|comission|
|  00000|2021-06-30|159632.82|      end|
|  00001|2021-05-17|   895.14|    begin|
|  00001|2021-05-17|    -1.17|comission|
|  00001|2021-05-17|  -146.45|      end|
|  00001|2021-07-31|     1.49|    begin|
|  00001|2021-07-31|     1.25|comission|
|  00001|2021-07-31| -6766.27|      end|
|  00002|2021-03-05|    -8.52|    begin|
|  00002|2021-03-05|    -0.92|comission|
|  00002|2021-03-05| -6706.29|      end|
|  00002|2021-03-23|     27.4|    begin|
|  00002|2021-03-23|   259.22|comission|
|  00002|2021-03-23|     4.06|      end|
|  00002|2021-05-09|-24527.77|    begin|
|  00002|2021-05-09| -1053.11|comission|
|  00002|2021-05-09| -4303.97|      end|
|  00004|2021-09-27|     1.28|    begin|
|  00004|2021-09-27|    -47.7|comission|
+-------+----------+---------+---------+
only showing top

Транспонируем изначачальный датасет с помощью метода pivot(), что позволит нам определять активность партнёра по формуле: begin.value + comission.value - end.value. Если значение вычисления равно нулю, значит в этот день партнёр был неактивен и эта дата нас не интересует.
Для метода pivot() требуется обернуть значения второй транспонируемой колонки в функцию агрегации, использование которой не даст нам нужный результат. Для решения этой проблемы сгруппируем и отсортируем наш датасет по id партнёров и датам отчёта:

In [ ]:
transposedDF = initialDF\
        .groupBy("partner", "rep_date")\
        .pivot("article").sum("value")\
        .where(F.col("begin")+F.col("comission")-F.col("end") != 0)\
        .orderBy("partner", "rep_date")

transposedDF.createOrReplaceTempView("transposed")

spark.sql("select * from transposed").show()

+-------+----------+---------+---------+---------+
|partner|  rep_date|    begin|comission|      end|
+-------+----------+---------+---------+---------+
|  00000|2021-06-30|    -0.11|     0.62|159632.82|
|  00001|2021-05-17|   895.14|    -1.17|  -146.45|
|  00001|2021-07-31|     1.49|     1.25| -6766.27|
|  00002|2021-03-05|    -8.52|    -0.92| -6706.29|
|  00002|2021-03-23|     27.4|   259.22|     4.06|
|  00002|2021-05-09|-24527.77| -1053.11| -4303.97|
|  00004|2021-09-27|     1.28|    -47.7|    11.64|
|  00004|2021-11-04|    -0.81|    31.84|   -32.65|
|  00005|2021-05-05|   112.61|     0.94|  1503.41|
|  00006|2021-11-01|  -812.79|  -699.48|  -113.31|
|  00006|2021-12-05|    -0.48|    30.83| -6491.63|
|  00006|2021-12-28|    79.82| 23844.79|     6.62|
|  00008|2021-01-28|148218.19|     1.55| -9552.59|
|  00008|2021-04-19|  -565.54|   5973.2| 50609.74|
|  00008|2021-11-01| -1538.83|    11.09|-79920.15|
|  00008|2021-11-28|  -130.77| -1202.58|    94.25|
|  00009|2021-01-02|    35.88| 

В этом запросе происходит следующее:

В подзапросе временной таблицы tmp мы отслеживаем активность партнёра. Для этого объединяем таблицу partner три раза, из этих таблиц вытягиваем значения баланса на момент begin, comission, end. Затем, если begin.value + com.value - end.value равно 0, то  клиент был неактивен и эта дата нам неинтересна, т.е. берём все оставшиеся.

Затем, в запросе временной таблицы мы выбираем предыдущие даты активности партнёра и нынешнюю дату активности из выше описанного запроса.

Затем из tmp мы берём идентификатор партнёра, дату репорта и проставляем значение жизни на основе разницы в днях (это не результирующая таблица).


In [ ]:
transferDF = spark.sql('''
with tmp as (
  select LAG(rep_date, 1, null) 
    Over (PARTITION BY partner ORDER BY rep_date) as prev_date, partner, rep_date
    from transposed
)

select partner, rep_date, 
  case 
    when DATEDIFF(day, prev_date, rep_date) >= 30 then 1
    else 0
  end as life
  from tmp 
  order by partner, rep_date
''')
transferDF.createOrReplaceTempView('transferDF')
transferDF.show(10)

+-------+----------+----+
|partner|  rep_date|life|
+-------+----------+----+
|  00000|2021-06-30|   0|
|  00001|2021-05-17|   0|
|  00001|2021-07-31|   1|
|  00002|2021-03-05|   0|
|  00002|2021-03-23|   0|
|  00002|2021-05-09|   1|
|  00004|2021-09-27|   0|
|  00004|2021-11-04|   1|
|  00005|2021-05-05|   0|
|  00006|2021-11-01|   0|
+-------+----------+----+
only showing top 10 rows



В следующем запросе происходит следующее:
Выбираем столбцы, которые нам нужны, а суммирование значения life мы оборачиваем в оконную функцию, которая разбивает на партиции по идентификаторам партнёра в порядке даты репорта и значения life.

Результирующая таблица выглядит так:


In [ ]:
spark.sql('''
select partner, rep_date, (sum(life) 
  over (partition by partner order by rep_date, life)) as life 
  from transferDF
''').show(50)

+-------+----------+----+
|partner|  rep_date|life|
+-------+----------+----+
|  00000|2021-06-30|   0|
|  00001|2021-05-17|   0|
|  00001|2021-07-31|   1|
|  00002|2021-03-05|   0|
|  00002|2021-03-23|   0|
|  00002|2021-05-09|   1|
|  00004|2021-09-27|   0|
|  00004|2021-11-04|   1|
|  00005|2021-05-05|   0|
|  00006|2021-11-01|   0|
|  00006|2021-12-05|   1|
|  00006|2021-12-28|   1|
|  00008|2021-01-28|   0|
|  00008|2021-04-19|   1|
|  00008|2021-11-01|   2|
|  00008|2021-11-28|   2|
|  00009|2021-01-02|   0|
|  00009|2021-05-25|   1|
|  00010|2021-04-26|   0|
|  00011|2021-09-07|   0|
|  00012|2021-01-27|   0|
|  00012|2021-02-22|   0|
|  00012|2021-04-05|   1|
|  00012|2021-05-02|   1|
|  00012|2021-11-30|   2|
|  00012|2021-12-05|   2|
|  00013|2021-04-25|   0|
|  00014|2021-02-19|   0|
|  00014|2021-12-26|   1|
|  00015|2021-06-09|   0|
|  00015|2021-09-21|   1|
|  00015|2021-11-06|   2|
|  00015|2021-11-08|   2|
|  00016|2021-02-26|   0|
|  00017|2021-02-25|   0|
|  00018|202